## 1. Import Data

In [1]:
import pickle
import gzip
import numpy as np
import os
import cv2

In [2]:
def load_zipped_pickle(filename):
    with gzip.open(filename, 'rb') as f:
        loaded_object = pickle.load(f)
        return loaded_object

In [3]:
def save_zipped_pickle(obj, filename):
    with gzip.open(filename, 'wb') as f:
        pickle.dump(obj, f, 2)

In [4]:
def evaluate(predictions, targets):
    ious = []
    for p, t in zip(predictions, targets):
        assert p['name'] == t['name']
        prediction = np.array(p['prediction'], dtype=bool)
        target = np.array(t['label'], dtype=bool)

        assert target.shape == prediction.shape
        overlap = prediction*target
        union = prediction + target

        ious.append(overlap.sum()/float(union.sum()))
    
    print("Median IOU: ", np.median(ious))

## 1.1 Load Data

In [5]:
# load data
train_data = load_zipped_pickle("train.pkl")
test_data = load_zipped_pickle("test.pkl")
samples = load_zipped_pickle("sample.pkl")



KeyboardInterrupt: 

数据结构：  
    train_data[1-65]包含65个病人的数据, 前46个是amateur数据， 后19个是expert数据  
    train_data[1]['video','label','dataset'...] , video是图片数组， label是MV mask，dataset代表是amateur还是expert  
    train_data[1]['video'] shape: [IMG_WIDTH, IMG_HEIGHT, FRAME NUM]， label也一样

In [ ]:
# test_img = train_data[60]['label'][:,:,24]
# zero_img = np.zeros(test_img.shape)
# zero_img[test_img] = 255
# zero_img_resize = cv2.resize(zero_img, (1000,1000))
# zero_img.shape

In [ ]:
# cv2.imshow("mask", zero_img_resize)
  
# cv2.waitKey(0) 

# cv2.destroyAllWindows() 

## 1.2 Data pre-processing

In [ ]:
# 样本图片大小
IMG_WIDTH = 256
IMG_HEIGHT = 256
IMG_CHANNELS = 1

In [ ]:
# 找到含有标记的帧
def is_labeled(mask_list):
    
    label_index = []
    for i in range((mask_list).shape[2]):
        if(True in mask_list[:,:,i]):
            label_index.append(i)
            continue
    return np.array(label_index)

def image_size_normalize(raw_data, size):
    #选出有标记的图片并resize到相应尺寸
    img_width, img_height, img_channel = size
    
    labeled_img_list = []
    mask_list = []
    
    for patient_i in range(len(raw_data)):

        img_i = raw_data[patient_i]['video'] # Shape: [IMG_WIDTH1, IMG_HEIGHT1, Frame_Num1]
        mask_i = raw_data[patient_i]['label']
        label_index = is_labeled(mask_i)
        labeled_img_i = img_i[:,:,label_index]
        labeled_mask_i = mask_i[:,:,label_index]
        
        for frame_j in range(labeled_mask_i.shape[2]):
            img_j = labeled_img_i[:,:,frame_j] # [IMG_WIDTH2, IMG_HEIGHT2, Frame_Num2]
            mask_j = labeled_mask_i[:,:,frame_j]
            mask_j_digi = np.zeros(mask_j.shape)
            mask_j_digi[mask_j] = 1
            if(img_j.shape[0]!= IMG_WIDTH or img_j.shape[1]!= IMG_HEIGHT):
                img_j = cv2.resize(img_j,(img_width, img_height))
                mask_j_digi = cv2.resize(mask_j_digi,(img_width, img_height))
            labeled_img_list.append(img_j)
            mask_list.append(mask_j_digi)
            
    return np.array(labeled_img_list).reshape(-1,img_width, img_height, img_channel),np.array(mask_list).reshape(-1,img_width, img_height, 1)

In [ ]:
# 从训练集中选出所有label过的图像并resize到指定大小
img_labeled, mask = image_size_normalize(train_data, (IMG_WIDTH,IMG_HEIGHT,IMG_CHANNELS))  # OutPut Shape: [Labeled_Frame_Num, IMG_WIDTH2, IMG_HEIGHT2, IMG_CHANNEL]

In [ ]:
img_labeled.shape

## 2 U-Net

## 2.1Model Definition 

In [ ]:
# for i in range(len(train_data)):
#     print(train_data[i]['dataset'])

In [ ]:
# for i in range(len(train_data)):
#     print(len(is_labeled(train_data[i]['label'])))


In [ ]:
import tensorflow as tf
from tensorflow import keras
import os
import numpy as np
from tqdm import tqdm
# from skimage.io import imread, imshow
# from skimage.transform import resize
import matplotlib.pyplot as plt
import random

In [ ]:
inputs = tf.keras.layers.Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)

In [ ]:
# UNet输入模块
def InputBlock(input, filters, kernel_size=3, strides=1, padding='same'):
    conv_1 = tf.keras.layers.Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding=padding,
                                    activation='relu')(input)  # 卷积块1
    return tf.keras.layers.Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding=padding,
                                  activation='relu')(conv_1)  # 卷积块2

def ContractingPathBlock(input, filters, kernel_size=3, strides=1, padding='same'):
    down_sampling = tf.keras.layers.MaxPool2D((2, 2))(input)  # 最大池化
    conv_1 = tf.keras.layers.Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding=padding,
                                    activation='relu')(down_sampling)  # 卷积块1
    return tf.keras.layers.Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding=padding,
                                    activation='relu')(conv_1)  # 卷积块2

# 扩张（恢复）路径模块
def ExpansivePathBlock(input, con_feature, filters, tran_filters, kernel_size=3, tran_kernel_size=2, strides=1,
                       tran_strides=2, padding='same', tran_padding='same'):
    upsampling = tf.keras.layers.Conv2DTranspose(filters=tran_filters, kernel_size=tran_kernel_size,
                                                 strides=tran_strides, padding=tran_padding)(input)  # 上采样（转置卷积方式）
    con_feature = tf.image.resize(con_feature, ((upsampling.shape)[1], (upsampling.shape)[2]),
                                  method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)  # 裁剪需要拼接的特征图
    concat_feature = tf.concat([con_feature, upsampling], axis=3)  # 拼接扩张层和收缩层的特征图（skip connection）
    conv_1 = tf.keras.layers.Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding=padding,
                                    activation='relu')(concat_feature)  # 卷积1
    return tf.keras.layers.Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding=padding,
                                  activation='relu')(conv_1)  # 卷积2

# UNet网络架构
def UNet(input_shape):
    inputs = tf.keras.layers.Input(input_shape)
    s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)
    
    # input block
    input_block = InputBlock(s, 64)

    # contracting path
    con_1 = ContractingPathBlock(input_block, 128)
    con_2 = ContractingPathBlock(con_1, 256)
    con_3 = ContractingPathBlock(con_2, 512)
    con_4 = ContractingPathBlock(con_3, 1024)

    # expansive path
    exp_4 = ExpansivePathBlock(con_4, con_3, 512, 512)
    exp_3 = ExpansivePathBlock(exp_4, con_2, 256, 256)
    exp_2 = ExpansivePathBlock(exp_3, con_1, 128, 128)
    exp_1 = ExpansivePathBlock(exp_2, input_block, 64, 64)

    outputs = tf.keras.layers.Conv2D(2, 1)(exp_1)  # 最终输出

    return tf.keras.Model(inputs=[inputs], outputs=[outputs])

## 2.2 Train

In [ ]:
model = UNet(input_shape=(IMG_WIDTH, IMG_HEIGHT, IMG_CHANNELS))
model.summary()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
results = model.fit(img_labeled, mask, batch_size=10, epochs=10)

# 3 Visualization

In [ ]:
import os

from PIL import Image

def video_make(name, image_list, mask_list,  img_size =(112, 112)):

    fps = 24 #视频每秒24帧
    size = img_size #需要转为视频的图片的尺寸

    #视频保存在当前目录下

    fourcc = cv2.VideoWriter_fourcc(*'MJPG')    
    video = cv2.VideoWriter(name+".avi",fourcc, fps,size, False)
    #draw stuff that goes on every frame here
    for frame_i in range(image_list.shape[2]):
        img = (image_list[:,:,frame_i])
        mask = mask_list[:,:,frame_i]
        img[mask] = 255
        img_mat = cv2.Mat(img)
        video.write(img)
    video.release()


## 4 Conclusion（things to be done for improvement）  
4.1 如上可见，随着epoch增加, accuracy波动很大，网络不是很work。 不清楚是数据量不够，网络复杂度不够， 还是iter比较小（纯ml小白）  
    如果是数据量不够， 或许需要用tracking的办法增加数据量
4.2 需要做cross-validation的预测版本
4.3 由于数据集中， 属于MV的像素点很少，和类别不平衡问题很像， 最好能增大 MV区域误识别的错误权重
4.4 预测的可视化实现